# Curl

Computing the curl of vector fields is a fundamental operation in vector calculus with applications in fluid dynamics, electromagnetism, and many other fields. This user-guide notebook showcases how to compute the curl of a vector field.

In [ ]:
import numpy as np

import uxarray as ux

## Curl Computation

### Background

The **curl** of a vector field **V** = (u, v) is a scalar field that measures the "rotation" or circulation density of the vector field at each point:

$$\text{curl}(\mathbf{V}) = \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}$$

**Physical Interpretation:**
- **Positive curl**: Indicates counterclockwise rotation or circulation
- **Negative curl**: Indicates clockwise rotation or circulation
- **Zero curl**: Indicates no local rotation (irrotational flow)

### Implementation

In UXarray, the curl is computed using the existing gradient infrastructure. The method leverages the finite-volume discretization to compute gradients of each vector component and then computes the cross-derivatives.

| **Input**                     |    **Usage**                    | **Output**                  |
| ----------------------------- | :-----------------------------: | --------------------------- |
| Vector field (u, v)           | `u.curl(v)`                     | Scalar field $\text{curl}(\mathbf{V})$ |

## Data

This notebook uses a subset of a 30km MPAS atmosphere grid, taken centered at 45 degrees longitude and 0 degrees latitude with a radius of 2 degrees.
- `face_lon`: Longitude at cell-centers
- `face_lat`: Latitude at cell-centers
- `gaussian`: Gaussian initialized at the center of the grid
- `inverse_gaussian`: Inverse of the gaussian above.

In [ ]:
base_path = "../../test/meshfiles/mpas/dyamond-30km/"
grid_path = base_path + "gradient_grid_subset.nc"
data_path = base_path + "gradient_data_subset.nc"

uxds = ux.open_dataset(grid_path, data_path)
print(f"Grid has {uxds.uxgrid.n_face} faces")
print(f"Available variables: {list(uxds.data_vars.keys())}")
uxds

## Usage

The curl method is available on `UxDataArray` objects and follows this signature:

```python
curl_result = u_component.curl(v_component)
```

The method returns a `UxDataArray` containing the curl values with the same shape and grid as the input components.

### Constant Fields (Mathematical Validation)

The curl of a constant vector field should be zero everywhere (within numerical precision). This provides an important validation of our implementation.

In [ ]:
# The curl of a constant vector field should be zero everywhere
u_constant = uxds["face_lat"] * 0 + 1.0  # Constant = 1
v_constant = uxds["face_lat"] * 0 + 2.0  # Constant = 2

# Compute curl
curl_constant = u_constant.curl(v_constant)

# Handle NaN values from boundary faces
finite_mask = np.isfinite(curl_constant.values)
finite_values = curl_constant.values[finite_mask]

print(f"Total faces: {len(curl_constant.values)}")
print(f"Finite values: {len(finite_values)}")
print(f"NaN values (boundary faces): {np.isnan(curl_constant.values).sum()}")

if len(finite_values) > 0:
    print(
        f"Finite curl range: [{finite_values.min():.10f}, {finite_values.max():.10f}]"
    )
    print(f"Maximum absolute curl (finite): {np.abs(finite_values).max():.2e}")
    print(f"Mean absolute curl (finite): {np.abs(finite_values).mean():.2e}")

    # Should be close to zero for constant field
    max_abs_curl = np.abs(finite_values).max()
    if max_abs_curl < 1e-10:
        print("✓ Curl is approximately zero as expected")
    else:
        print(f"⚠ Curl is {max_abs_curl:.2e} (may be due to discretization)")

### Rotational Fields

Let's create a simple rotational vector field to demonstrate non-zero curl.

In [ ]:
# Create a simple rotational field: u = -y, v = x (simplified)
# Using lat/lon as proxies for x/y coordinates
u_rotational = -uxds["face_lat"]  # u = -y
v_rotational = uxds["face_lon"]  # v = x

# Compute curl
curl_rotational = u_rotational.curl(v_rotational)

# Handle NaN values from boundary faces
finite_mask = np.isfinite(curl_rotational.values)
finite_values = curl_rotational.values[finite_mask]

print(f"Total faces: {len(curl_rotational.values)}")
print(f"Interior faces: {len(finite_values)}")
print(f"Boundary faces: {np.isnan(curl_rotational.values).sum()}")

if len(finite_values) > 0:
    print(f"Finite curl range: [{finite_values.min():.6f}, {finite_values.max():.6f}]")
    print(f"Mean curl (finite): {finite_values.mean():.6f}")
    print(f"Standard deviation: {finite_values.std():.6f}")

### Gaussian Fields

In [ ]:
# Use Gaussian fields as vector components
u_gauss = uxds["gaussian"]
v_gauss = uxds["inverse_gaussian"]

# Compute curl
curl_gauss = u_gauss.curl(v_gauss)

# Handle NaN values from boundary faces
finite_mask = np.isfinite(curl_gauss.values)
finite_values = curl_gauss.values[finite_mask]

print(f"Total faces: {len(curl_gauss.values)}")
print(f"Interior faces: {len(finite_values)}")
print(f"Boundary faces: {np.isnan(curl_gauss.values).sum()}")

if len(finite_values) > 0:
    print(f"Finite curl range: [{finite_values.min():.6f}, {finite_values.max():.6f}]")
    print(f"Mean curl (finite): {finite_values.mean():.6f}")

## Vector Calculus Identity: Curl of Gradient

An important vector calculus identity states that the curl of a gradient is zero: $\nabla \times (\nabla \phi) = 0$ for any scalar field $\phi$.

This provides a mathematical validation of our curl implementation.

In [ ]:
# Compute gradient of the Gaussian field
grad_gaussian = uxds["gaussian"].gradient()

# Extract gradient components
grad_u = grad_gaussian["zonal_gradient"]
grad_v = grad_gaussian["meridional_gradient"]

# Compute curl of gradient (should be approximately zero)
curl_of_gradient = grad_u.curl(grad_v)

# Analyze results
finite_mask = np.isfinite(curl_of_gradient.values)
finite_curl = curl_of_gradient.values[finite_mask]

print("Curl of gradient computed successfully!")
print(f"Interior faces: {len(finite_curl)}")
print(f"Boundary faces: {np.isnan(curl_of_gradient.values).sum()}")

if len(finite_curl) > 0:
    print(f"Curl of gradient range: [{finite_curl.min():.6f}, {finite_curl.max():.6f}]")
    print(f"Mean curl of gradient: {finite_curl.mean():.6f}")
    print(f"Maximum absolute curl: {np.abs(finite_curl).max():.2e}")

    # Check if it's approximately zero (within numerical precision)
    max_abs_curl = np.abs(finite_curl).max()
    if max_abs_curl < 1e-6:
        print("✓ Identity curl(∇φ) ≈ 0 verified within numerical precision")
    else:
        print(f"⚠ Identity not perfectly satisfied (max error: {max_abs_curl:.2e})")
        print("  This is expected due to discrete computation on unstructured grids")